In [ ]:
import pandas as pd

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
dataset_url = "https://raw.githubusercontent.com/Scherpinski-R/Brain-Stroke-Prediction/main/BrainStrokeData/full_data.csv"

In [ ]:
df = pd.read_csv(dataset_url)

y = df["stroke"]
X = df.loc[:, df.columns!="stroke"]

In [ ]:
categorical_features = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]

In [ ]:
numerical_features = ["age", "avg_glucose_level", "bmi"]

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
columns_trans = ColumnTransformer(
    transformers=[
    ("cat", OneHotEncoder(), categorical_features),
    ("num", StandardScaler(), numerical_features)], 
    remainder='passthrough'
)

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
pipe = make_pipeline(columns_trans, SVC())

In [ ]:
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['gender', 'ever_married',
                                                   'work_type',
                                                   'Residence_type',
                                                   'smoking_status']),
                                                 ('num', StandardScaler(),
                                                  ['age', 'avg_glucose_level',
                                                   'bmi'])])),
                ('svc', SVC())])

In [ ]:
 X_new = X.head()
 X_new

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked


In [ ]:
pipe.predict(X_new)

array([0, 0, 0, 0, 0])